# Author: [Ruslan Brilenkov](https://www.linkedin.com/in/ruslan-brilenkov/)

# [Original article]() 

## Importing necessary libraries/packages

In [1]:
import numpy as np
import re
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
# from keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers import Adam, RMSprop # - Works
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

## If working on Google Colaboratory [Colab](https://medium.datadriveninvestor.com/free-gpu-for-deep-learning-776a178c6ebf)

In [4]:
working_on_colab = False
if working_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import os

    dir_to_drive_Colab = "/content/drive/MyDrive/Files_from_Colab/LSTM_Alice_in_Wonderland/"
    os.listdir(dir_to_drive_Colab)

Mounted at /content/drive


In [9]:
os.system("mkdir ./data/")

if working_on_colab:
    os.system(f"cp -r {dir_to_drive_Colab}data/* ./data/")
else:
    print("Upload the data by hand")

0

## Decide if we are training the model from scratch and/or loading the pre-trained model

In [10]:
load_saved_model = False
train_model = True

# The first step is to clean up and [tokenize](https://medium.datadriveninvestor.com/a-few-notes-about-text-tokenization-c5c67635638d) the text.

Tokenization is the process of split‐ ting the text up into individual units, such as words or characters.

In [11]:
token_type = 'word'

In [12]:
# Part 1. Loading the text and performing some cleanup!
# our data file contaning "Alice in Wonderland"
filename = "./data/data.txt"

# opening the file
with open(filename, encoding='utf-8-sig') as f:
    text = f.read()

# if you wish to read text beforehand, uncomment:
# text

# Part 2. Removing text before and after the main stories 
start = text.find("CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n")
end = text.find("\n\n                             THE END")
text = text[start:end]

# Printing out the result:
text

'CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n  Alice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!  Oh dear!  I shall be late!\'  (when she thought\nit over afterwards, it occurred to her that she ought to have\nwondered at this, but at the time it all seemed quite natural);\nbu

In [13]:
print(len(text))

148329


In [14]:
# text

In [15]:
# Step 3. Separating every chapter:
seq_length = 20 
start_story = '~ ' * seq_length
# adding the first chapter name back:
text = start_story + text

# Step 4. lowering the case
text = text.lower()
# separating every chapter with distinguished symbols for a better training
text = text.replace('\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

text

' ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ chapter i . down the rabbit - hole . alice was beginning to get very tired of sitting by her sister on the bank , and of having nothing to do : . once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ` and what is the use of a book , \' thought alice ` without pictures or conversation ? \' . so she was considering in her own mind ( as well as she could , for the hot day made her feel very sleepy and stupid ) , whether the pleasure of making a daisy - chain would be worth the trouble of getting up and picking the daisies , when suddenly a white rabbit with pink eyes ran close by her . there was nothing so very remarkable in that ; nor did alice think it so very much out of the way to hear the rabbit say to itself , ` oh dear ! . oh dear ! . i shall be late ! \' . ( when she thought it over afterwards , it occurred to her that she ought to have wondered at this , but at the time it all

In [16]:
print(len(text))
text

152801


' ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ chapter i . down the rabbit - hole . alice was beginning to get very tired of sitting by her sister on the bank , and of having nothing to do : . once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ` and what is the use of a book , \' thought alice ` without pictures or conversation ? \' . so she was considering in her own mind ( as well as she could , for the hot day made her feel very sleepy and stupid ) , whether the pleasure of making a daisy - chain would be worth the trouble of getting up and picking the daisies , when suddenly a white rabbit with pink eyes ran close by her . there was nothing so very remarkable in that ; nor did alice think it so very much out of the way to hear the rabbit say to itself , ` oh dear ! . oh dear ! . i shall be late ! \' . ( when she thought it over afterwards , it occurred to her that she ought to have wondered at this , but at the time it all

In [22]:
if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

# printing interesting quntities:
print(f"Number of tokenized words: {total_words}")
# the mapping dictionary between words and indices
print(tokenizer.word_index)
# text after tokenization
print(token_list)

Number of tokenized words: 2656
{',': 1, '.': 2, 'the': 3, '`': 4, "'": 5, 'and': 6, 'to': 7, '-': 8, 'a': 9, 'she': 10, 'it': 11, 'of': 12, 'said': 13, '!': 14, 'i': 15, 'alice': 16, 'in': 17, 'you': 18, 'was': 19, 'that': 20, 'as': 21, '~': 22, 'her': 23, ':': 24, 'at': 25, '?': 26, ';': 27, 'on': 28, 'all': 29, 'with': 30, 'had': 31, 'but': 32, 'for': 33, 'so': 34, 'be': 35, 'very': 36, 'not': 37, 'what': 38, 'this': 39, 'they': 40, 'little': 41, 'he': 42, 'out': 43, '"': 44, 'is': 45, 'one': 46, 'down': 47, 'up': 48, 'if': 49, 'his': 50, 'then': 51, 'about': 52, 'no': 53, 'them': 54, 'know': 55, 'like': 56, 'were': 57, 'would': 58, 'went': 59, 'again': 60, 'herself': 61, 'do': 62, 'have': 63, 'when': 64, 'or': 65, 'could': 66, 'there': 67, 'thought': 68, 'off': 69, 'time': 70, 'how': 71, 'queen': 72, 'into': 73, 'see': 74, 'me': 75, 'well': 76, 'did': 77, 'your': 78, 'who': 79, "don't": 80, 'king': 81, 'now': 82, '*': 83, "i'm": 84, 'by': 85, 'began': 86, 'my': 87, 'its': 88, 'an':

# Building our dataset

### Our LSTM network will be trained to predict the next word in a sequence, given a sequence of words preceding this point.

### The parameter of our training process (X) is a sequence length which we use for model training

### While, the response variable (Y) for each sequence is the subsequent/next word. We use one-hot encoding into a vector of length 2656 (the number of distinct words in the vocabulary)

In [39]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    
    # one-hot encoding, creating a categorical variable:
    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

# printing output:
print(f"Inout shape: {X.shape}")
print(f"Output shape: {y.shape}")


Number of sequences: 35564 

Inout shape: (35564, 20)
Output shape: (35564, 2656)


In [35]:
print(len(token_list))
print(len(token_list) - seq_length)

35584
35564


(35564, 20)

(35564, 2656)

# [LSTM architecture](https://medium.datadriveninvestor.com/a-story-generator-using-lstm-inside-recurrent-neural-network-rnn-f823b295571d)

## Define LSTM model

In [36]:
if load_saved_model:
    model = load_model('./saved_models/lr=0.001.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    # x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)
    learning_rate = 0.001
    opti = RMSprop(learning_rate = learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


## Fitting the model

In [38]:

epochs = 100
batch_size = 32
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)


Epoch 1/100
1112/1112 [==============================] - 117s 103ms/step - loss: 5.3206
Epoch 2/100
1112/1112 [==============================] - 113s 102ms/step - loss: 4.8221
Epoch 3/100
1112/1112 [==============================] - 112s 101ms/step - loss: 4.6278
Epoch 4/100
1112/1112 [==============================] - 113s 102ms/step - loss: 4.4827
Epoch 5/100
1112/1112 [==============================] - 116s 104ms/step - loss: 4.3775
Epoch 6/100
1112/1112 [==============================] - 118s 106ms/step - loss: 4.2786
Epoch 7/100
1112/1112 [==============================] - 116s 104ms/step - loss: 4.2159
Epoch 8/100
1112/1112 [==============================] - 114s 102ms/step - loss: 4.1114
Epoch 9/100
1112/1112 [==============================] - 114s 103ms/step - loss: 4.0057
Epoch 10/100
1112/1112 [==============================] - 109s 98ms/step - loss: 3.9031
Epoch 11/100
1112/1112 [==============================] - 112s 101ms/step - loss: 3.8196
Epoch 12/100
1112/1112 [=======

In [44]:

# One way to save the model, in h5 format
os.system("mkdir ./saved_models/")

# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save(f'./saved_models/lr={learning_rate}.h5')

if working_on_colab:
    os.system(f"cp -r ./saved_models/lr={learning_rate}.h5 {dir_to_drive_Colab}/")


# Another way to save the model, in TensorFlow format directly
os.system("mkdir ./saved_models/")
# saving the model in tensorflow format
model.save(f'./lr={learning_rate}_tf', save_format='tf')
if working_on_colab:
    os.system(f"cp -r ./lr={learning_rate}_tf {dir_to_drive_Colab}/")

# loading the saved model
loaded_model = load_model(f'./lr={learning_rate}_tf')

# retraining the model if needed, etc. ...
loaded_model.summary()


INFO:tensorflow:Assets written to: ./lr=0.001_tf/assets


INFO:tensorflow:Assets written to: ./lr=0.001_tf/assets


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


In [45]:
# creating the folder called saved_models
os.system("mkdir ./saved_models/")

# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
# saving the model in .h5 format
model.save(f'./saved_models/lr={learning_rate}.h5')

# loading this model:
new_model = load_model(f'./saved_models/lr={learning_rate}.h5')
# checking the summary:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('./') 

# Generating text

In [46]:

def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "~":
            break
            
        if token_type == 'word':
            output_text += output_word + " "
            seed_text += output_word + " "
        else:
            output_text += output_word + " " 
            seed_text += output_word + " "
            
    return output_text


In [47]:
seed_text = ""
gen_words = 500

print('Temp 0.2')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
print('Temp 0.33')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
print('Temp 0.5')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))


Temp 0.2
. chapter . for a an here . ` a queen ! . and are for head ! ' alice but she off at the from to herself , ` but you could have little of them , ' and got on ` the they ? ' such a white rabbit . ` i turtle with the was their but back the was in alice ; ' she found herself in won't can the little his said . ` the first back upon all the cat , and went on to went in a what mouse it . however , added tell a little " and the what was to see that it a little i had no " would be added and for . and what ? ' . ` i were when some king , ' the hatter said to the did , you know . ` no , what ; the gryphon will not go she on the then , ` and that it cat then , i know , and i think , ' . alice dormouse to went again , and went on in a with its shall things come on , and him by this ; and another very an when she thought her , the march hare had you had ? ' . ` it had the queen gave them , ' oh the king think , and alice , anxiously very curious round it , and look at the door again won't b

In [43]:
print('Temp 1.0')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

Temp 1.0
~ . chapter to hastily . down and in a at great : . she found out of a little came , and a this your alice to if it all isn't say , and looked at she never got them again , and at last in very thing have sight of half very that's a time but they wonder never tried . alice ran his three with the i gave that the left ! . off being a in the : . the is that a out of hand , and old like way is , . by how don't have it knew that ? ' . ` this - - why tell me , ' alice rather went i , and an his i'll tried that ) . ` and that i'm been when enough , up and other i know ! . your or you what about all the window ? ' . ` dance for shouldn't you in my walked ? ' said alice , much very much a sure when so look down her large , in the he : . he he for the long a hurried turtle , ' and the could wouldn't go on . alice king it looked go great must have shall the however , which won't added to a up in two of good sort ! . once didn't it would . ' . ` i could and air , now , ' said the king ; tw

# The end of this step-by-step guide for generating novel stories using Keras LSTM RNN architecture.